Ref: https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/machine-learning-model-predictions-ml-net

## 0. Add & Import packages

In [22]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.DataView"
#r "nuget: Microsoft.ML.OnnxRuntime"
#r "nuget: Microsoft.ML.OnnxTransformer"

Installed Packages Microsoft.ML, 4.0.2 Microsoft.ML.DataView, 4.0.2 Microsoft.ML.OnnxRuntime, 1.23.1 Microsoft.ML.OnnxTransformer, 4.0.2

In [23]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms.Onnx;

In [ ]:
public class HousingOnnxInput
{
    [ColumnName("Size")]
    public float Size { get; set; }

    [ColumnName("HistoricalPrices")]
    public float[] HistoricalPrices { get; set; }
}

public class HousingOnnxOutput
{
    [ColumnName("Score.output")]
    public float PredictedPrice { get; set; }
}
var inputColumns = new string[]
{
    "Size", "HistoricalPrices"
};

var outputColumns = new string[] { "Score" };

//Create MLContext
MLContext mlContext = new MLContext();
var onnxPredictionPipeline =
    mlContext
        .Transforms
        .ApplyOnnxModel(
            "../ExportedModels/onnx_model.onnx");
var emptyDv = mlContext.Data.LoadFromEnumerable(new HousingOnnxInput[] { });
var onnxPredictionPipelineTransformer = onnxPredictionPipeline.Fit(emptyDv);

//Load Trained Model
var onnxPredictionEngine = mlContext.Model.CreatePredictionEngine<HousingOnnxInput, HousingOnnxOutput>(onnxPredictionPipelineTransformer);
var input = new HousingOnnxInput
{
    Size = 1000f,
    HistoricalPrices = new float[] { 100000f, 125000f, 122000f }
};
var prediction = onnxPredictionEngine.Predict(input);

// Predicted Price
Console.WriteLine($"Predicted Price: {prediction.PredictedPrice}");

## 1. Create data models
### a. Input data

In [11]:
public class HousingData
{
    [LoadColumn(0)]
    public float Size { get; set; }

    [LoadColumn(1, 3)]
    [VectorType(3)]
    public float[] HistoricalPrices { get; set; }

    [LoadColumn(4)]
    [ColumnName("Label")]
    public float CurrentPrice { get; set; }
}

### b. Output data
Like the Features and Label input column names, ML.NET has default names for the predicted value columns produced by a model. The names can differ depending on the task.
Because the algorithm used in this sample is a linear regression algorithm, the default name of the output column is Score, which is defined by the ColumnName attribute on the PredictedPrice property.

In [12]:
class HousingPrediction
{
    [ColumnName("Score")]
    public float PredictedPrice { get; set; }
}

## 2. Set up a prediction pipeline
Whether making a single or batch prediction, the prediction pipeline needs to be loaded into the application. This pipeline contains both the data preprocessing transformations and the trained model. The following code snippet loads the prediction pipeline from a file named model.zip.

In [13]:
//Create MLContext
MLContext mlContext = new MLContext();

// Load Trained Model
DataViewSchema predictionPipelineSchema;
ITransformer predictionPipeline = mlContext.Model.Load("../ExportedModels/model.zip", out predictionPipelineSchema);

### a. Single prediction
To make a single prediction, create a PredictionEngine using the loaded prediction pipeline.

In [14]:
// Create PredictionEngines
PredictionEngine<HousingData, HousingPrediction> predictionEngine = mlContext.Model.CreatePredictionEngine<HousingData, HousingPrediction>(predictionPipeline);

Then, use the Predict method and pass in your input data as a parameter. Notice that using the Predict method doesn't require the input to be an IDataView. That's because it conveniently internalizes the input data type manipulation so you can pass in an object of the input data type. Additionally, since CurrentPrice is the target or label you're trying to predict using new data, it's assumed there's no value for it at the moment.

In [15]:
// Input Data
HousingData inputData = new HousingData
{
    Size = 900f,
    HistoricalPrices = new float[] { 155000f, 190000f, 220000f }
};

// Get Prediction
HousingPrediction prediction = predictionEngine.Predict(inputData);
Console.WriteLine($"PredictedPrice: {prediction.PredictedPrice}");

PredictedPrice: 206645.23


** Tip
- PredictionEngine is not thread-safe. Additionally, you have to create an instance of it everywhere it is needed within your application. As your application grows, this process can become unmanageable. For improved performance and thread safety, use a combination of dependency injection and the PredictionEnginePool service, which creates an ObjectPool of PredictionEngine objects for use throughout your application.
- For examples on how to use the PredictionEnginePool service, see deploy a model to a web API and deploy a model to Azure Functions.
- For more information, see Dependency injection in ASP.NET Core.

### b. Multiple predictions (IDataView)
Given the following data, load it into an IDataView. In this case, the name of the IDataView is inputData. Because CurrentPrice is the target or label you're trying to predict using new data, it's assumed there's no value for it at the moment.

In [16]:
// Actual data
HousingData[] housingData = new HousingData[]
{
    new HousingData
    {
        Size = 850f,
        HistoricalPrices = new float[] { 150000f, 175000f, 210000f }
    },
    new HousingData
    {
        Size = 900f,
        HistoricalPrices = new float[] { 155000f, 190000f, 220000f }
    },
    new HousingData
    {
        Size = 550f,
        HistoricalPrices = new float[] { 99000f, 98000f, 130000f }
    }
};

Then, use the Transform method to apply the data transformations and generate predictions.

In [17]:
// Convert array to IDataView
IDataView inputDataView = mlContext.Data.LoadFromEnumerable(housingData);

// Predicted Data
IDataView predictions = predictionPipeline.Transform(inputDataView);

Inspect the predicted values by using the GetColumn method.

In [18]:
// Get Predictions
float[] scoreColumn = predictions.GetColumn<float>("Score").ToArray();

foreach(var x in scoreColumn){
    Console.WriteLine($"PredictedPrice: {x}");
}

PredictedPrice: 201589.25
PredictedPrice: 206645.23
PredictedPrice: 167148.55


### c. Multiple predictions (PredictionEnginePool)
To make multiple predictions using PredictionEnginePool, you can take an IEnumerable containing multiple instances of your model input. For example, take an IEnumerable<HousingInput> and apply the Predict method to each element using the LINQ Select method.

This code sample assumes you have a PredictionEnginePool called predictionEnginePool and an IEnumerable<HousingData> called housingData.

In [19]:
// Program.cs
services.AddPredictionEnginePool<HousingData, HousingPrediction>()
        .FromFile(modelName: "housingModel", filePath: "model.zip", watchForChanges: true);

Error: (2,1): error CS0103: The name 'services' does not exist in the current context

In [20]:
private readonly PredictionEnginePool<HousingData, HousingPrediction> _predictionEnginePool;
public MyController(PredictionEnginePool<HousingData, HousingPrediction> pool)
{
    _predictionEnginePool = pool;
}
IEnumerable<HousingPrediction> predictions = housingData.Select(input => predictionEnginePool.Predict(input));

Error: (2,8): error CS1520: Method must have a return type

### d. Prediction with ONNX

In [21]:
public class HousingOnnxInput
{
    [ColumnName("Size")]
    public float Size { get; set; }

    [ColumnName("HistoricalPrices")]
    public float[] HistoricalPrices { get; set; }
}

public class HousingOnnxOutput
{
    [ColumnName("Score.output")]
    public float PredictedPrice { get; set; }
}

In [ ]:
var inputColumns = new string[]
{
    "Size", "HistoricalPrices"
};

var outputColumns = new string[] { "Score" };

//Create MLContext
MLContext mlContext = new MLContext();
var onnxPredictionPipeline =
    mlContext
        .Transforms
        .ApplyOnnxModel(
            outputColumnNames: outputColumns,
            inputColumnNames: inputColumns,
            "../ExportedModels/onnx_model.onnx");
var emptyDv = mlContext.Data.LoadFromEnumerable(new HousingOnnxInput[] { });
var onnxPredictionPipelineTransformer = onnxPredictionPipeline.Fit(emptyDv);

//Load Trained Model
var onnxPredictionEngine = mlContext.Model.CreatePredictionEngine<HousingOnnxInput, HousingOnnxOutput>(onnxPredictionPipelineTransformer);
var input = new HousingOnnxInput
{
    Size = 1000f,
    HistoricalPrices = new float[] { 100000f, 125000f, 122000f }
};
var prediction = onnxPredictionEngine.Predict(input);

// Predicted Price
Console.WriteLine($"Predicted Price: {prediction.PredictedPrice}");